In [2]:
import pandas as pd
import numpy as np
import json
import requests
import plotly.express as px
import matplotlib.pyplot as plt
import datetime
import time
from bs4 import BeautifulSoup
from bs4 import BeautifulStoneSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import tempfile
import time
from bs4 import BeautifulSoup

In [2]:
patients=pd.read_csv('patients.csv')
treatment=pd.read_csv('treatments.csv')
adverse=pd.read_csv('adverse_reactions.csv')
treatment_cuts=pd.read_csv('treatments_cut.csv')

In [14]:
patients[patients.duplicated(subset=['given_name','surname'])]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi


In [5]:
# with pd.ExcelWriter("clinical_trials.xlsx") as writer:
#     patients.to_excel(writer,sheet_name='patients')
#     treatment.to_excel(writer,sheet_name='treatments')
#     treatment_cuts.to_excel(writer,sheet_name='treatment_cut')
#     adverse.to_excel(writer,sheet_name='adverse_reaction')

In [13]:
patients.drop_duplicates(subset=['given_name', 'surname'], inplace=True)


In [22]:
patients_df=patients.copy()
treatment_df=treatment.copy()
adverse_df=adverse.copy()
treatment_cuts_df=treatment_cuts.copy()

In [32]:
patients_df[patients_df['address']=='No data']

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
209,210,female,Lalita,Eldarkhanov,No data,No data,No data,No data,No data,No data,8/14/1950,143.4,62,26.2
219,220,male,Mỹ,Quynh,No data,No data,No data,No data,No data,No data,4/9/1978,237.8,69,35.1
230,231,female,Elisabeth,Knudsen,No data,No data,No data,No data,No data,No data,9/23/1976,165.9,63,29.4
234,235,female,Martina,Tománková,No data,No data,No data,No data,No data,No data,4/7/1936,199.5,65,33.2
242,243,male,John,O'Brian,No data,No data,No data,No data,No data,No data,2/25/1957,205.3,74,26.4
249,250,male,Benjamin,Mehler,No data,No data,No data,No data,No data,No data,10/30/1951,146.5,69,21.6
257,258,male,Jin,Kung,No data,No data,No data,No data,No data,No data,5/17/1995,231.7,69,34.2
264,265,female,Wafiyyah,Asfour,No data,No data,No data,No data,No data,No data,11/3/1989,158.6,63,28.1
269,270,female,Flavia,Fiorentino,No data,No data,No data,No data,No data,No data,10/9/1937,175.2,61,33.1
278,279,female,Generosa,Cabán,No data,No data,No data,No data,No data,No data,12/16/1962,124.3,69,18.4


In [28]:
patients_df.fillna(str("No data"),inplace=True)

In [35]:
treatment_df['hba1c_change']=treatment_df['hba1c_start']-treatment_df['hba1c_end']
treatment_df

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,0.43
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.47
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,0.43
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32
...,...,...,...,...,...,...,...
275,albina,zetticci,45u - 51u,-,7.93,7.73,0.20
276,john,teichelmann,-,49u - 49u,7.90,7.58,0.32
277,mathea,lillebø,23u - 36u,-,9.04,8.67,0.37
278,vallie,prince,31u - 38u,-,7.64,7.28,0.36


In [36]:
treatment_cuts_df['hba1c_change']=treatment_cuts_df['hba1c_start']-treatment_cuts_df['hba1c_end']
treatment_cuts_df

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,jožka,resanovič,22u - 30u,-,7.56,7.22,0.34
1,inunnguaq,heilmann,57u - 67u,-,7.85,7.45,0.40
2,alwin,svensson,36u - 39u,-,7.78,7.34,0.44
3,thể,lương,-,61u - 64u,7.64,7.22,0.42
4,amanda,ribeiro,36u - 44u,-,7.85,7.47,0.38
...,...,...,...,...,...,...,...
65,rovzan,kishiev,32u - 37u,-,7.75,7.41,0.34
66,jakob,jakobsen,-,28u - 26u,7.96,7.51,0.45
67,bernd,schneider,48u - 56u,-,7.74,7.44,0.30
68,berta,napolitani,-,42u - 44u,7.68,7.21,0.47


In [39]:
treatment_cuts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    70 non-null     object 
 1   surname       70 non-null     object 
 2   auralin       70 non-null     object 
 3   novodra       70 non-null     object 
 4   hba1c_start   70 non-null     float64
 5   hba1c_end     70 non-null     float64
 6   hba1c_change  70 non-null     float64
dtypes: float64(3), object(4)
memory usage: 4.0+ KB


In [44]:
patients_df['contact'].str.extract(r'(\d{3}[\s,]?\d{3}[\s,]?\d{4})')


,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
498,NaN
499,NaN
500,NaN
501,360 443 2060


In [42]:
patients_df['contact'].str.split("-")


0               [951, 719, 9170ZoeWellish@superrito.com]
1               [PamelaSHill@cuvox.de+1 (217) 569, 3204]
2                   [402, 363, 6804JaeMDebord@gustr.com]
3          [PhanBaLiem@jourrapide.com+1 (732) 636, 8246]
4                    [334, 515, 7487TimNeudorf@cuvox.de]
                             ...                        
498      [207, 477, 0579MustafaLindstrom@jourrapide.com]
499               [928, 284, 4492RumanBisliev@gustr.com]
500            [816, 223, 6007JinkedeKeizer@teleworm.us]
501    [ChidaluOnyekaozulu@jourrapide.com1 360 443 2060]
502             [PatrickGersten@rhyta.com402, 848, 4923]
Name: contact, Length: 498, dtype: object

In [110]:
treatments_df=pd.concat([treatment_df,treatment_cuts_df])

In [111]:
treatments_df=treatments_df.melt(id_vars=['given_name','surname','hba1c_start','hba1c_end','hba1c_change'])
treatments_df

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,variable,value
0,veronika,jindrová,7.63,7.20,0.43,auralin,41u - 48u
1,elliot,richardson,7.56,7.09,0.47,auralin,-
2,yukitaka,takenaka,7.68,7.25,0.43,auralin,-
3,skye,gormanston,7.97,7.62,0.35,auralin,33u - 36u
4,alissa,montez,7.78,7.46,0.32,auralin,-
...,...,...,...,...,...,...,...
695,rovzan,kishiev,7.75,7.41,0.34,novodra,-
696,jakob,jakobsen,7.96,7.51,0.45,novodra,28u - 26u
697,bernd,schneider,7.74,7.44,0.30,novodra,-
698,berta,napolitani,7.68,7.21,0.47,novodra,42u - 44u


In [112]:
treatments_df=treatments_df.rename(columns={'variable':'Medicine_type','value':'dosage_range'})
treatments_df=treatments_df[treatments_df['dosage_range']!='-']
treatments_df['dosage_range']=treatments_df['dosage_range'].str.replace('u','')

treatments_df['dosage_start_range']=treatments_df['dosage_range'].str.split('-').str.get(0)

treatments_df['dosage_end_range']=treatments_df['dosage_range'].str.split('-').str.get(1)

treatments_df.drop(columns='dosage_range',inplace=True)
treatments_df.sort_values(by='given_name')



,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Medicine_type,dosage_start_range,dosage_end_range
609,aaliyah,rice,7.64,7.33,0.31,novodra,31,31
446,abdul-nur,isa,7.98,7.53,0.45,novodra,54,50
449,abel,yonatan,7.88,7.50,0.38,novodra,38,39
368,addolorata,lombardi,7.75,7.33,0.42,novodra,49,46
565,adib,ghanem,7.59,7.12,0.47,novodra,46,49
...,...,...,...,...,...,...,...,...
657,zoe,wellish,7.71,7.30,0.41,novodra,33,33
390,ásta,grímsdóttir,7.62,7.16,0.46,novodra,29,30
566,þórunn,tryggvadóttir,7.98,7.51,0.47,novodra,30,32
325,štefanija,novosel,7.64,7.17,0.47,auralin,37,42


In [113]:
treatments_df['dosage_start_range']=treatments_df['dosage_start_range'].astype('int')

treatments_df['dosage_end_range']=treatments_df['dosage_end_range'].astype('int')
treatments_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 350 entries, 0 to 698
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   given_name          350 non-null    object 
 1   surname             350 non-null    object 
 2   hba1c_start         350 non-null    float64
 3   hba1c_end           350 non-null    float64
 4   hba1c_change        350 non-null    float64
 5   Medicine_type       350 non-null    object 
 6   dosage_start_range  350 non-null    int32  
 7   dosage_end_range    350 non-null    int32  
dtypes: float64(3), int32(2), object(3)
memory usage: 21.9+ KB


In [114]:
treatments_df.head()

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Medicine_type,dosage_start_range,dosage_end_range
0,veronika,jindrová,7.63,7.20,0.43,auralin,41,48
3,skye,gormanston,7.97,7.62,0.35,auralin,33,36
6,sophia,haugen,7.65,7.27,0.38,auralin,37,42
7,eddie,archer,7.89,7.55,0.34,auralin,31,38
9,asia,woźniak,7.76,7.37,0.39,auralin,30,36


In [115]:
adverse_df.head()

,given_name,surname,adverse_reaction
0,berta,napolitani,injection site discomfort
1,lena,baer,hypoglycemia
2,joseph,day,hypoglycemia
3,flavia,fiorentino,cough
4,manouck,wubbels,throat irritation


In [116]:
treatments_df=treatments_df.merge(adverse_df,how='left',on=['given_name','surname'])


In [117]:
treatments_df[treatments_df['given_name']=='Dsvid']

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Medicine_type,dosage_start_range,dosage_end_range,adverse_reaction
